<a href="https://colab.research.google.com/github/martanascimento1/datascience1/blob/main/decisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Decision Tree:
1- classification;
2- regression;



In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np

Loading the database:

In [ ]:
df_diabetes = pd.read_csv('/home/marta/datascience/data/data2/diabetes.csv')


In [ ]:
df_diabetes.head()

In [ ]:
df_diabetes.info()

Dividing the data into training and testing:

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(df_diabetes.drop('Outcome', axis=1), df_diabetes['Outcome'],test_size=0.3)

Checking the shape of the data:

In [ ]:
x_train.shape,x_test.shape

In [ ]:
y_train.shape,y_test.shape

Instantiating the classifier object:

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()

Classifier parameters:


*   Criterion: division quality measure
*   splitter: strategy used to split the decision node
*   max_depth: maximum tree depth
*   min_samples_split: minimum number of samples to consider a node for splitting
*   min_samples_leaf: number of minimum samples at leaf level



Training the decision tree model:

In [ ]:
clf = clf.fit(x_train,y_train)

Checking the most important features for the trained model:

In [ ]:
clf.feature_importances_

In [ ]:

clf = clf.fit(x_train, y_train)
for feature, importancia in zip(df_diabetes.columns, clf.feature_importances_):
    print("{}={}".format(feature, importancia))

In [ ]:
resultado = clf.predict(x_test)
resultado

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test, resultado))

Rendering the tree graphically:

In [ ]:
!pip install ipywidgets
!pip install graphviz
!pip install pydot

In [ ]:
import pydot
import graphviz

In [ ]:
dot_data = export_graphviz(
         clf,
         out_file=None,
         feature_names = df_diabetes.drop('Outcome', axis=1).columns,
         class_names=['0','1'],
         filled=True, rounded=True,
         proportion=True,
         node_ids=True,
         rotate=True,
         label='all',
         special_characters=True
)
graph = graphviz.Source(dot_data)
graph

rendering the tree interactively:

In [ ]:
from ipywidgets import interactive
from IPython.display import  SVG,display
from graphviz import Source
from sklearn.tree import export_graphviz

In [ ]:

x, y = df_diabetes.drop('Outcome', axis=1), df_diabetes['Outcome']

features_label = df_diabetes.drop('Outcome', axis=1).columns
class_label = ['0','1']

def plot_tree(crit, split, depth, min_samples_split, min_samples_leaf=0.2):
    estimator = DecisionTreeClassifier(
        random_state=0,
        criterion=crit,
        splitter=split,
        max_depth=depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf
    )

    estimator.fit(x, y)
    graph = Source(export_graphviz(estimator,
      out_file=None,
      feature_names=features_label,
      class_names=class_label, # Added missing comma
      impurity = True,
      filled = True ))
    display(SVG(graph.pipe(format='svg')))
    return estimator

inter = interactive(plot_tree, # Moved interactive call outside of the function
                   crit = ['gini', 'entropy'],
                   split = ['best', 'random'],
                   depth=[1,2,3,4,5,10,20,30],
                   min_samples_split=(1, 5),
                   min_samples_leaf=(1, 5))

display(inter) # Now correctly indented outside the function

Visualizing the borders created by the tree:

In [ ]:
!pip install mlxtend
from mlxtend.plotting import plot_decision_regions
import matplotlib.pyplot as plt

In [ ]:
def vizualize_fronteiras(min_samples_split,max_depth):
    x = df_diabetes [['Glucose', 'Insulin']].values
    y = df_diabetes.Outcome.values
    clf = DecisionTreeClassifier(min_samples_split=min_samples_split, max_depth=max_depth)
    tree = clf.fit(x, y)
    plt.figure(figsize=(16, 9))
    plot_decision_regions(x, y, clf=tree, legend=2)
    plt.xlabel('Glucose')
    plt.ylabel('Insulin')
    plt.title('Decision Tree')
    plt.show()

In [ ]:
vizualize_fronteiras(2, max_depth=30)

In [ ]:
vizualize_fronteiras(10, max_depth=8)

Navigating the decision tree structure:

In [ ]:
estimator = DecisionTreeClassifier(max_depth=3, min_samples_split =2, min_samples_leaf=2)
estimator.fit(x, y)

In [ ]:
n_nodes = estimator.tree_.node_count
children_left = estimator.tree_.children_left
children_right = estimator.tree_.children_right
feature = estimator.tree_.feature
threshold = estimator.tree_.threshold

In [ ]:
print("numero de nós: {}".format(n_nodes))